In [1]:
# All imports
import numpy as np
import matplotlib.pyplot as plt

In [4]:
start_price = 131
# percentage volatility or annualized volatility
vola = 0.25
# Overall trend or average return per year
drift = 0.35 
# Total number of simulations
n_simul = 1000 
# Assumption 252 trading days
delta = 1.0 / 252.0
# Runtime in days
runtime = 252 * 5

In [6]:
def brownian_motion(delta, vola, runtime, n_simul):
    # standard deviation or scale is sqrt of volatiliy for brownian motion
    return vola * np.random.normal(loc=0, scale=np.sqrt(delta),
                                   size=(runtime, n_simul))

In [7]:
# Random geometric motion returns
def rgm_returns(delta, vola, runtime, drift, n_simul):
    motion = brownian_motion(delta, vola, runtime, n_simul)
    return np.exp(motion + (drift - vola ** 2 / 2) * delta)

In [8]:
def rgm_levels(start_price, delta, vola, runtime, drift, n_simul):
    returns = rgm_returns(delta, vola, runtime, drift, n_simul)
    stacked = np.vstack([np.ones(n_simul), returns]) # prepends 1 infront of all returns
    return start_price * stacked.cumprod(axis=0)

In [9]:
stock_prices = rgm_levels(start_price, delta, 
                          vola, runtime, drift, 
                          n_simul)

In [ ]:
plt.plot(stock_prices, linewidth=0.25)
plt.show()